In [242]:
import requests
from bs4 import BeautifulSoup
import re
import uuid
import csv
import pandas as pd

In [243]:
df = pd.read_csv('fighter_links_UUID.csv')
df.head()

,link,UUID
0,http://www.ufcstats.com/fighter-details/c32aeb...,7e60e735-5d1d-4fa3-b7f7-bcc4a1f50c04
1,http://www.ufcstats.com/fighter-details/23024f...,cec78d13-b105-4e36-a908-2bcd35d211c9
2,http://www.ufcstats.com/fighter-details/480779...,84d443f9-78b9-4e4e-9b86-bd38270ccad1
3,http://www.ufcstats.com/fighter-details/94426b...,9f4d943c-191c-4a56-a20c-4286297f581b
4,http://www.ufcstats.com/fighter-details/49d2a0...,abbdfe77-12e0-40e4-b3dd-30bc906a5d71


In [244]:
df2 = pd.read_csv('event_UUID.csv')
df2.head()

,UUID,name,date,location,link
0,09f61ec0-1b0a-4c7d-9c99-90e20a0ae45d,UFC Fight Night: Burns vs. Morales,"May 17, 2025","Las Vegas, Nevada, USA",http://www.ufcstats.com/event-details/8ad022dd...
1,66a2d9bc-de67-43bf-87ee-b3db1b689797,UFC 315: Muhammad vs. Della Maddalena,"May 10, 2025","Montreal, Quebec, Canada",http://www.ufcstats.com/event-details/118463dd...
2,700f1515-41ab-4bdc-9a4f-7484296e3eaf,UFC Fight Night: Sandhagen vs. Figueiredo,"May 03, 2025","Des Moines, Iowa, USA",http://www.ufcstats.com/event-details/de20ffb3...
3,b931be39-e4dc-43b4-9942-88b108bb6007,UFC Fight Night: Machado Garry vs. Prates,"April 26, 2025","Kansas City, Missouri, USA",http://www.ufcstats.com/event-details/b2e3aca4...
4,52898274-33fa-49f0-adbf-77d912de1518,UFC 314: Volkanovski vs. Lopes,"April 12, 2025","Miami, Florida, USA",http://www.ufcstats.com/event-details/22f4b6cb...


In [245]:
columns = [
    'fighter_id', 'corner', 'name', 'alias', 'opponent_id', 'event_name', 'event_date', 'method_of_victory', 'winner_id',
    'height', 'weight', 'reach', 'stance', 'dOB', 'rounds_format', 'round_end', 'time_end', 'referee', 'r1_knockdowns',
    'r1_sig_strikes_landed', 'r1_sig_strikes_attempted', 'r1_total_strikes_landed', 'r1_total_strikes_attempted',
    'r1_takedowns_landed', 'r1_takedowns_attempted', 'r1_subs_attempted', 'r1_reversals', 'r1_control_time',
    'r1_head_sig_strikes_landed', 'r1_head_sig_strikes_attempted', 'r1_body_sig_strikes_landed', 'r1_body_sig_strikes_attempted',
    'r1_leg_sig_strikes_landed', 'r1_leg_sig_strikes_attempted', 'r1_distance_sig_strikes_landed', 'r1_distance_sig_strikes_attempted',
    'r1_clinch_sig_strikes_landed', 'r1_clinch_sig_strikes_attempted', 'r1_ground_sig_strikes_landed', 'r1_ground_sig_strikes_attempted',
    'r2_knockdowns', 'r2_sig_strikes_landed', 'r2_sig_strikes_attempted', 'r2_total_strikes_landed', 'r2_total_strikes_attempted',
    'r2_takedowns_landed', 'r2_takedowns_attempted', 'r2_subs_attempted', 'r2_reversals', 'r2_control_time',
    'r2_head_sig_strikes_landed', 'r2_head_sig_strikes_attempted', 'r2_body_sig_strikes_landed', 'r2_body_sig_strikes_attempted',
    'r2_leg_sig_strikes_landed', 'r2_leg_sig_strikes_attempted', 'r2_distance_sig_strikes_landed', 'r2_distance_sig_strikes_attempted',
    'r2_clinch_sig_strikes_landed', 'r2_clinch_sig_strikes_attempted', 'r2_ground_sig_strikes_landed', 'r2_ground_sig_strikes_attempted',
    'r3_knockdowns', 'r3_sig_strikes_landed', 'r3_sig_strikes_attempted', 'r3_total_strikes_landed', 'r3_total_strikes_attempted',
    'r3_takedowns_landed', 'r3_takedowns_attempted', 'r3_subs_attempted', 'r3_reversals', 'r3_control_time',
    'r3_head_sig_strikes_landed', 'r3_head_sig_strikes_attempted', 'r3_body_sig_strikes_landed', 'r3_body_sig_strikes_attempted',
    'r3_leg_sig_strikes_landed', 'r3_leg_sig_strikes_attempted', 'r3_distance_sig_strikes_landed', 'r3_distance_sig_strikes_attempted',
    'r3_clinch_sig_strikes_landed', 'r3_clinch_sig_strikes_attempted', 'r3_ground_sig_strikes_landed', 'r3_ground_sig_strikes_attempted',
    'r4_knockdowns', 'r4_sig_strikes_landed', 'r4_sig_strikes_attempted', 'r4_total_strikes_landed', 'r4_total_strikes_attempted',
    'r4_takedowns_landed', 'r4_takedowns_attempted', 'r4_subs_attempted', 'r4_reversals', 'r4_control_time',
    'r4_head_sig_strikes_landed', 'r4_head_sig_strikes_attempted', 'r4_body_sig_strikes_landed', 'r4_body_sig_strikes_attempted',
    'r4_leg_sig_strikes_landed', 'r4_leg_sig_strikes_attempted', 'r4_distance_sig_strikes_landed', 'r4_distance_sig_strikes_attempted',
    'r4_clinch_sig_strikes_landed', 'r4_clinch_sig_strikes_attempted', 'r4_ground_sig_strikes_landed', 'r4_ground_sig_strikes_attempted',
    'r5_knockdowns', 'r5_sig_strikes_landed', 'r5_sig_strikes_attempted', 'r5_total_strikes_landed', 'r5_total_strikes_attempted',
    'r5_takedowns_landed', 'r5_takedowns_attempted', 'r5_subs_attempted', 'r5_reversals', 'r5_control_time',
    'r5_head_sig_strikes_landed', 'r5_head_sig_strikes_attempted', 'r5_body_sig_strikes_landed', 'r5_body_sig_strikes_attempted',
    'r5_leg_sig_strikes_landed', 'r5_leg_sig_strikes_attempted', 'r5_distance_sig_strikes_landed', 'r5_distance_sig_strikes_attempted',
    'r5_clinch_sig_strikes_landed', 'r5_clinch_sig_strikes_attempted', 'r5_ground_sig_strikes_landed', 'r5_ground_sig_strikes_attempted'
]

dfCheck = pd.DataFrame(columns=columns)
dfCheck.head()

,fighter_id,corner,name,alias,opponent_id,event_name,event_date,method_of_victory,winner_id,height,...,r5_body_sig_strikes_landed,r5_body_sig_strikes_attempted,r5_leg_sig_strikes_landed,r5_leg_sig_strikes_attempted,r5_distance_sig_strikes_landed,r5_distance_sig_strikes_attempted,r5_clinch_sig_strikes_landed,r5_clinch_sig_strikes_attempted,r5_ground_sig_strikes_landed,r5_ground_sig_strikes_attempted


In [246]:
events_UUID = df2['name'].values
opponent_keys = df.set_index(df.columns[0])[df.columns[1]].to_dict()
links = df['link'].values

In [247]:
url = 'http://www.ufcstats.com/fighter-details/35673bf5204524ee'
fighter_id = opponent_keys[url]
site = requests.get(url)
soup = BeautifulSoup(site.content, 'html.parser')
print(site)
print(fighter_id)

<Response [200]>
f0a63a6f-5af0-47af-b263-bb4b1e27f698


In [248]:
height = None
reach = None
stance = None
dOB = None

details = soup.find('ul', class_='b-list__box-list')
lis = details.find_all('li')
for li in lis:
    desc = li.text.strip().split(':', 1)

    if desc[0].lower() == 'height':
        height = desc[1].lstrip()
    elif desc[0].lower() == 'reach':
        reach = desc[1].lstrip()
    elif desc[0].lower() == 'stance':
        stance = desc[1].lstrip()
    elif desc[0].lower() == 'dob':
        dOB = desc[1].lstrip()

print(height)
print(reach)
print(stance)
print(dOB)


6' 1"
76"
Southpaw
Jan 14, 1996


In [249]:
fights = soup.find_all('tr', class_=re.compile("b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click"))
fight_links = []
for fight in fights:
    fight_links.append(fight.find('a')['href'])
print(len(fight_links))

7


In [250]:
events = soup.find_all('td', class_=re.compile('b-fight-details__table-col l-page_align_left'))
print(len(events))

21


In [251]:
while len(fight_links)*3 != len(events):
    events.pop(0)
print(len(events))

21


In [252]:
for i in range(0, len(fight_links)):
    #initalized before fight link
    opponent_id = None
    event_name = None
    event_date = None
    method_of_victory = None
    details = events[i*3:i*3+3]

    event_info = details[1].find_all('p', class_='b-fight-details__table-text')
    event_name = event_info[0].text.strip()
    event_date = event_info[1].text.strip().replace('.', '')

    if event_name not in events_UUID:
        continue


    opponent_link = details[0].find_all('a')[1]['href']
    opponent_id = opponent_keys[opponent_link]

    method_info = details[2].find_all('p', class_='b-fight-details__table-text')
    method_of_victory = f'{method_info[0].text.strip()}  {method_info[1].text.strip()}'


    name = None
    alias = None
    winner_id = None
    corner = None
    weight = None
    round_end = None
    time_end = None
    time_format = None
    referee = None
    
    url = fight_links[i]
    site = requests.get(url)
    soup = BeautifulSoup(site.content, 'html.parser')
    
    details = soup.find_all('div', class_='b-fight-details__person')
    red_corner_info = details[0].find('a', class_='b-link b-fight-details__person-link')
    blue_corner_info = details[1].find('a', class_='b-link b-fight-details__person-link')

    #fighter is in red corner
    if opponent_keys[red_corner_info['href']] == fighter_id:
        corner = 'Red'
        name = red_corner_info.text.strip()
        alias = details[0].find('p', class_='b-fight-details__person-title').text.strip().replace('"', '')
        if details[0].find('i').text.strip() == "W":
            winner_id = fighter_id
        elif details[1].find('i').text.strip() =="W":
            winner_id = opponent_id
    #fighter is in blue corner
    else:
        corner = 'Blue'
        name = blue_corner_info.text.strip()
        alias = details[1].find('p', class_='b-fight-details__person-title').text.strip().replace('"', '')
        if details[1].find('i').text.strip() == "W":
            winner_id = fighter_id
        elif details[0].find('i').text.strip() =="W":
            winner_id = opponent_id


    details = soup.find('div', class_='b-fight-details__fight')
    weight = details.find('i', class_='b-fight-details__fight-title').text.strip().replace(' Bout', '')


    labels = details.find_all('i', class_='b-fight-details__text-item')
    for label in labels:
        desc = label.text.strip().split(':', 1)
        if desc[0].lower() == 'round':
            round_end = desc[1].lstrip()
        elif desc[0].lower() == 'time':
            time_end = desc[1].lstrip()
        elif desc[0].lower() == 'time format':
            time_format = desc[1].lstrip()
        elif desc[0].lower() == 'referee':
            referee = desc[1].lstrip()


    tables = soup.find_all('table', class_='b-fight-details__table js-fight-table')
    headers = tables[0].find_all('th', class_='b-fight-details__table-col')
    statistics = tables[0].find_all('td', class_='b-fight-details__table-col')
    ss_statistics = tables[1].find_all('td', class_='b-fight-details__table-col')

    header_str = []
    for header in headers:
        header_str.append(header.text.lstrip().rstrip().replace('\n', ''))
    header_str[5] = header_str[5].replace(' %', '')
    header_list = header_str[0:10]
    rounds_breakdown = header_str[10:]

    stat_list = []
    for stat in statistics:
        stat_list.append(stat.text.replace('\n', '').lstrip().rstrip())

    ss_stat_list = []
    for stat in ss_statistics:
        ss_stat_list.append(stat.text.replace('\n', '').lstrip().rstrip())

    #Initalize totals
    r1_knockdowns = None
    r1_sig_strikes_landed = None
    r1_sig_strikes_attempted = None
    r1_total_strikes_landed = None
    r1_total_strikes_attempted = None
    r1_takedowns_landed = None
    r1_takedowns_attempted = None
    r1_subs_attempted = None
    r1_reversals = None
    r1_control_time = None

    r2_knockdowns = None
    r2_sig_strikes_landed = None
    r2_sig_strikes_attempted = None
    r2_total_strikes_landed = None
    r2_total_strikes_attempted = None
    r2_takedowns_landed = None
    r2_takedowns_attempted = None
    r2_subs_attempted = None
    r2_reversals = None
    r2_control_time = None

    r3_knockdowns = None
    r3_sig_strikes_landed = None
    r3_sig_strikes_attempted = None
    r3_total_strikes_landed = None
    r3_total_strikes_attempted = None
    r3_takedowns_landed = None
    r3_takedowns_attempted = None
    r3_subs_attempted = None
    r3_reversals = None
    r3_control_time = None

    r4_knockdowns = None
    r4_sig_strikes_landed = None
    r4_sig_strikes_attempted = None
    r4_total_strikes_landed = None
    r4_total_strikes_attempted = None
    r4_takedowns_landed = None
    r4_takedowns_attempted = None
    r4_subs_attempted = None
    r4_reversals = None
    r4_control_time = None

    r5_knockdowns = None
    r5_sig_strikes_landed = None
    r5_sig_strikes_attempted = None
    r5_total_strikes_landed = None
    r5_total_strikes_attempted = None
    r5_takedowns_landed = None
    r5_takedowns_attempted = None
    r5_subs_attempted = None
    r5_reversals = None
    r5_control_time = None



    #significant strikes
    r1_head_sig_strikes_landed = None
    r1_head_sig_strikes_attempted = None
    r1_body_sig_strikes_landed = None
    r1_body_sig_strikes_attempted = None
    r1_leg_sig_strikes_landed = None
    r1_leg_sig_strikes_attempted = None
    r1_distance_sig_strikes_landed = None
    r1_distance_sig_strikes_attempted = None
    r1_clinch_sig_strikes_attempted = None
    r1_clinch_sig_strikes_landed = None
    r1_ground_sig_strikes_landed = None
    r1_ground_sig_strikes_attempted = None

    r2_head_sig_strikes_landed = None
    r2_head_sig_strikes_attempted = None
    r2_body_sig_strikes_landed = None
    r2_body_sig_strikes_attempted = None
    r2_leg_sig_strikes_landed = None
    r2_leg_sig_strikes_attempted = None
    r2_distance_sig_strikes_landed = None
    r2_distance_sig_strikes_attempted = None
    r2_clinch_sig_strikes_attempted = None
    r2_clinch_sig_strikes_landed = None
    r2_ground_sig_strikes_landed = None
    r2_ground_sig_strikes_attempted = None

    r3_head_sig_strikes_landed = None
    r3_head_sig_strikes_attempted = None
    r3_body_sig_strikes_landed = None
    r3_body_sig_strikes_attempted = None
    r3_leg_sig_strikes_landed = None
    r3_leg_sig_strikes_attempted = None
    r3_distance_sig_strikes_landed = None
    r3_distance_sig_strikes_attempted = None
    r3_clinch_sig_strikes_attempted = None
    r3_clinch_sig_strikes_landed = None
    r3_ground_sig_strikes_landed = None
    r3_ground_sig_strikes_attempted = None

    r4_head_sig_strikes_landed = None
    r4_head_sig_strikes_attempted = None
    r4_body_sig_strikes_landed = None
    r4_body_sig_strikes_attempted = None
    r4_leg_sig_strikes_landed = None
    r4_leg_sig_strikes_attempted = None
    r4_distance_sig_strikes_landed = None
    r4_distance_sig_strikes_attempted = None
    r4_clinch_sig_strikes_attempted = None
    r4_clinch_sig_strikes_landed = None
    r4_ground_sig_strikes_landed = None
    r4_ground_sig_strikes_attempted = None

    r5_head_sig_strikes_landed = None
    r5_head_sig_strikes_attempted = None
    r5_body_sig_strikes_landed = None
    r5_body_sig_strikes_attempted = None
    r5_leg_sig_strikes_landed = None
    r5_leg_sig_strikes_attempted = None
    r5_distance_sig_strikes_landed = None
    r5_distance_sig_strikes_attempted = None
    r5_clinch_sig_strikes_attempted = None
    r5_clinch_sig_strikes_landed = None
    r5_ground_sig_strikes_landed = None
    r5_ground_sig_strikes_attempted = None


    for rounds in rounds_breakdown:
        if corner == "Red":
            pullI = 0
        else:
            pullI = 1

        if rounds == 'Round 1':
            rd1_stats = stat_list[0:10]

            r1_knockdowns = rd1_stats[1].split(' ', 1)[pullI].lstrip().rstrip()

            sig_str = rd1_stats[2].split('  ')[pullI*5].split(' of ')
            r1_sig_strikes_landed = sig_str[0]
            r1_sig_strikes_attempted = sig_str[1]


            total_str = rd1_stats[4].split('  ')[pullI*5].split(' of ')
            r1_total_strikes_landed = total_str[0]
            r1_total_strikes_attempted = total_str[1]


            total_td = rd1_stats[5].split('  ')[pullI*5].split(' of ')
            r1_takedowns_landed = total_td[0]
            r1_takedowns_attempted = total_td[1]

            r1_subs_attempted = rd1_stats[7].split(' ', 1)[pullI].lstrip().rstrip()

            r1_reversals = rd1_stats[8].split(' ', 1)[pullI].lstrip().rstrip()

            r1_control_time = rd1_stats[9].split(' ', 1)[pullI].lstrip().rstrip()


            #get sig strike divs
            rd1_ss_stats = ss_stat_list[0:9]

            rd1_head_total = rd1_ss_stats[3].split('  ')[pullI*5].split(' of ')
            r1_head_sig_strikes_landed = rd1_head_total[0]
            r1_head_sig_strikes_attempted = rd1_head_total[1]


            rd1_body_total = rd1_ss_stats[4].split('  ')[pullI*5].split(' of ')
            r1_body_sig_strikes_landed = rd1_body_total[0]
            r1_body_sig_strikes_attempted = rd1_body_total[1]

            rd1_leg_total = rd1_ss_stats[5].split('  ')[pullI*5].split(' of ')
            r1_leg_sig_strikes_landed = rd1_leg_total[0]
            r1_leg_sig_strikes_attempted = rd1_leg_total[1]

            rd1_distance_total = rd1_ss_stats[6].split('  ')[pullI*5].split(' of ')
            r1_distance_sig_strikes_landed = rd1_distance_total[0]
            r1_distance_sig_strikes_attempted = rd1_distance_total[1]

            rd1_clinch_total = rd1_ss_stats[7].split('  ')[pullI*5].split(' of ')
            r1_clinch_sig_strikes_landed = rd1_clinch_total[0]
            r1_clinch_sig_strikes_attempted = rd1_clinch_total[1]

            rd1_ground_total = rd1_ss_stats[8].split('  ')[pullI*5].split(' of ')
            r1_ground_sig_strikes_landed = rd1_ground_total[0]
            r1_ground_sig_strikes_attempted = rd1_ground_total[1]

            

        elif rounds == 'Round 2':
            rd2_stats = stat_list[10:20]

            r2_knockdowns = rd2_stats[1].split(' ', 1)[pullI].lstrip().rstrip()

            sig_str = rd2_stats[2].split('  ')[pullI*5].split(' of ')
            r2_sig_strikes_landed = sig_str[0]
            r2_sig_strikes_attempted = sig_str[1]


            total_str = rd2_stats[4].split('  ')[pullI*5].split(' of ')
            r2_total_strikes_landed = total_str[0]
            r2_total_strikes_attempted = total_str[1]


            total_td = rd2_stats[5].split('  ')[pullI*5].split(' of ')
            r2_takedowns_landed = total_td[0]
            r2_takedowns_attempted = total_td[1]

            r2_subs_attempted = rd2_stats[7].split(' ', 1)[pullI].lstrip().rstrip()

            r2_reversals = rd2_stats[8].split(' ', 1)[pullI].lstrip().rstrip()

            r2_control_time = rd2_stats[9].split(' ', 1)[pullI].lstrip().rstrip() 


            #get sig strike divs
            rd2_ss_stats = ss_stat_list[9:18]

            rd2_head_total = rd2_ss_stats[3].split('  ')[pullI*5].split(' of ')
            r2_head_sig_strikes_landed = rd2_head_total[0]
            r2_head_sig_strikes_attempted = rd2_head_total[1]


            rd2_body_total = rd2_ss_stats[4].split('  ')[pullI*5].split(' of ')
            r2_body_sig_strikes_landed = rd2_body_total[0]
            r2_body_sig_strikes_attempted = rd2_body_total[1]

            rd2_leg_total = rd2_ss_stats[5].split('  ')[pullI*5].split(' of ')
            r2_leg_sig_strikes_landed = rd2_leg_total[0]
            r2_leg_sig_strikes_attempted = rd2_leg_total[1]

            rd2_distance_total = rd2_ss_stats[6].split('  ')[pullI*5].split(' of ')
            r2_distance_sig_strikes_landed = rd2_distance_total[0]
            r2_distance_sig_strikes_attempted = rd2_distance_total[1]

            rd2_clinch_total = rd2_ss_stats[7].split('  ')[pullI*5].split(' of ')
            r2_clinch_sig_strikes_landed = rd2_clinch_total[0]
            r2_clinch_sig_strikes_attempted = rd2_clinch_total[1]

            rd2_ground_total = rd2_ss_stats[8].split('  ')[pullI*5].split(' of ')
            r2_ground_sig_strikes_landed = rd2_ground_total[0]
            r2_ground_sig_strikes_attempted = rd2_ground_total[1]

        
        elif rounds == 'Round 3':
            rd3_stats = stat_list[20:30]

            r3_knockdowns = rd3_stats[1].split(' ', 1)[pullI].lstrip().rstrip()

            sig_str = rd3_stats[2].split('  ')[pullI*5].split(' of ')
            r3_sig_strikes_landed = sig_str[0]
            r3_sig_strikes_attempted = sig_str[1]


            total_str = rd3_stats[4].split('  ')[pullI*5].split(' of ')
            r3_total_strikes_landed = total_str[0]
            r3_total_strikes_attempted = total_str[1]


            total_td = rd3_stats[5].split('  ')[pullI*5].split(' of ')
            r3_takedowns_landed = total_td[0]
            r3_takedowns_attempted = total_td[1]

            r3_subs_attempted = rd3_stats[7].split(' ', 1)[pullI].lstrip().rstrip()

            r3_reversals = rd3_stats[8].split(' ', 1)[pullI].lstrip().rstrip()

            r3_control_time = rd3_stats[9].split(' ', 1)[pullI].lstrip().rstrip() 


            #get sig strike divs
            rd3_ss_stats = ss_stat_list[18:27]

            rd3_head_total = rd3_ss_stats[3].split('  ')[pullI*5].split(' of ')
            r3_head_sig_strikes_landed = rd3_head_total[0]
            r3_head_sig_strikes_attempted = rd3_head_total[1]


            rd3_body_total = rd3_ss_stats[4].split('  ')[pullI*5].split(' of ')
            r3_body_sig_strikes_landed = rd3_body_total[0]
            r3_body_sig_strikes_attempted = rd3_body_total[1]

            rd3_leg_total = rd3_ss_stats[5].split('  ')[pullI*5].split(' of ')
            r3_leg_sig_strikes_landed = rd3_leg_total[0]
            r3_leg_sig_strikes_attempted = rd3_leg_total[1]

            rd3_distance_total = rd3_ss_stats[6].split('  ')[pullI*5].split(' of ')
            r3_distance_sig_strikes_landed = rd3_distance_total[0]
            r3_distance_sig_strikes_attempted = rd3_distance_total[1]

            rd3_clinch_total = rd3_ss_stats[7].split('  ')[pullI*5].split(' of ')
            r3_clinch_sig_strikes_landed = rd3_clinch_total[0]
            r3_clinch_sig_strikes_attempted = rd3_clinch_total[1]

            rd3_ground_total = rd3_ss_stats[8].split('  ')[pullI*5].split(' of ')
            r3_ground_sig_strikes_landed = rd3_ground_total[0]
            r3_ground_sig_strikes_attempted = rd3_ground_total[1]


        elif rounds == 'Round 4':
            rd4_stats = stat_list[30:40]

            r4_knockdowns = rd4_stats[1].split(' ', 1)[pullI].lstrip().rstrip()

            sig_str = rd4_stats[2].split('  ')[pullI*5].split(' of ')
            r4_sig_strikes_landed = sig_str[0]
            r4_sig_strikes_attempted = sig_str[1]


            total_str = rd4_stats[4].split('  ')[pullI*5].split(' of ')
            r4_total_strikes_landed = total_str[0]
            r4_total_strikes_attempted = total_str[1]


            total_td = rd4_stats[5].split('  ')[pullI*5].split(' of ')
            r4_takedowns_landed = total_td[0]
            r4_takedowns_attempted = total_td[1]

            r4_subs_attempted = rd4_stats[7].split(' ', 1)[pullI].lstrip().rstrip()

            r4_reversals = rd4_stats[8].split(' ', 1)[pullI].lstrip().rstrip()

            r4_control_time = rd4_stats[9].split(' ', 1)[pullI].lstrip().rstrip()  


            #get sig strike divs
            rd4_ss_stats = ss_stat_list[27:36]

            rd4_head_total = rd4_ss_stats[3].split('  ')[pullI*5].split(' of ')
            r4_head_sig_strikes_landed = rd4_head_total[0]
            r4_head_sig_strikes_attempted = rd4_head_total[1]


            rd4_body_total = rd4_ss_stats[4].split('  ')[pullI*5].split(' of ')
            r4_body_sig_strikes_landed = rd4_body_total[0]
            r4_body_sig_strikes_attempted = rd4_body_total[1]

            rd4_leg_total = rd4_ss_stats[5].split('  ')[pullI*5].split(' of ')
            r4_leg_sig_strikes_landed = rd4_leg_total[0]
            r4_leg_sig_strikes_attempted = rd4_leg_total[1]

            rd4_distance_total = rd4_ss_stats[6].split('  ')[pullI*5].split(' of ')
            r4_distance_sig_strikes_landed = rd4_distance_total[0]
            r4_distance_sig_strikes_attempted = rd4_distance_total[1]

            rd4_clinch_total = rd4_ss_stats[7].split('  ')[pullI*5].split(' of ')
            r4_clinch_sig_strikes_landed = rd4_clinch_total[0]
            r4_clinch_sig_strikes_attempted = rd4_clinch_total[1]

            rd4_ground_total = rd4_ss_stats[8].split('  ')[pullI*5].split(' of ')
            r4_ground_sig_strikes_landed = rd4_ground_total[0]
            r4_ground_sig_strikes_attempted = rd4_ground_total[1]



        elif rounds == 'Round 5':
            rd5_stats = stat_list[40:50]

            r5_knockdowns = rd5_stats[1].split(' ', 1)[pullI].lstrip().rstrip()

            sig_str = rd5_stats[2].split('  ')[pullI*5].split(' of ')
            r5_sig_strikes_landed = sig_str[0]
            r5_sig_strikes_attempted = sig_str[1]


            total_str = rd5_stats[4].split('  ')[pullI*5].split(' of ')
            r5_total_strikes_landed = total_str[0]
            r5_total_strikes_attempted = total_str[1]


            total_td = rd5_stats[5].split('  ')[pullI*5].split(' of ')
            r5_takedowns_landed = total_td[0]
            r5_takedowns_attempted = total_td[1]

            r5_subs_attempted = rd5_stats[7].split(' ', 1)[pullI].lstrip().rstrip()

            r5_reversals = rd5_stats[8].split(' ', 1)[pullI].lstrip().rstrip()

            r5_control_time = rd5_stats[9].split(' ', 1)[pullI].lstrip().rstrip()     


            #get sig strike divs
            rd5_ss_stats = ss_stat_list[36:45]

            rd5_head_total = rd5_ss_stats[3].split('  ')[pullI*5].split(' of ')
            r5_head_sig_strikes_landed = rd5_head_total[0]
            r5_head_sig_strikes_attempted = rd5_head_total[1]


            rd5_body_total = rd5_ss_stats[4].split('  ')[pullI*5].split(' of ')
            r5_body_sig_strikes_landed = rd5_body_total[0]
            r5_body_sig_strikes_attempted = rd5_body_total[1]

            rd5_leg_total = rd5_ss_stats[5].split('  ')[pullI*5].split(' of ')
            r5_leg_sig_strikes_landed = rd5_leg_total[0]
            r5_leg_sig_strikes_attempted = rd5_leg_total[1]

            rd5_distance_total = rd5_ss_stats[6].split('  ')[pullI*5].split(' of ')
            r5_distance_sig_strikes_landed = rd5_distance_total[0]
            r5_distance_sig_strikes_attempted = rd5_distance_total[1]

            rd5_clinch_total = rd5_ss_stats[7].split('  ')[pullI*5].split(' of ')
            r5_clinch_sig_strikes_landed = rd5_clinch_total[0]
            r5_clinch_sig_strikes_attempted = rd5_clinch_total[1]

            rd5_ground_total = rd5_ss_stats[8].split('  ')[pullI*5].split(' of ')
            r5_ground_sig_strikes_landed = rd5_ground_total[0]
            r5_ground_sig_strikes_attempted = rd5_ground_total[1]   

    performance = [fighter_id, corner, name, alias, opponent_id, event_name, event_date, method_of_victory, winner_id, 
        height, weight, reach, stance, dOB, time_format, round_end, time_end, referee, r1_knockdowns,
        r1_sig_strikes_landed, r1_sig_strikes_attempted, r1_total_strikes_landed, r1_total_strikes_attempted,
        r1_takedowns_landed, r1_takedowns_attempted, r1_subs_attempted, r1_reversals, r1_control_time,
        r1_head_sig_strikes_landed, r1_head_sig_strikes_attempted, r1_body_sig_strikes_landed, r1_body_sig_strikes_attempted,
        r1_leg_sig_strikes_landed, r1_leg_sig_strikes_attempted, r1_distance_sig_strikes_landed, r1_distance_sig_strikes_attempted,
        r1_clinch_sig_strikes_landed, r1_clinch_sig_strikes_attempted, r1_ground_sig_strikes_landed, r1_ground_sig_strikes_attempted,
        r2_knockdowns, r2_sig_strikes_landed, r2_sig_strikes_attempted, r2_total_strikes_landed, r2_total_strikes_attempted,
        r2_takedowns_landed, r2_takedowns_attempted, r2_subs_attempted, r2_reversals, r2_control_time,
        r2_head_sig_strikes_landed, r2_head_sig_strikes_attempted, r2_body_sig_strikes_landed, r2_body_sig_strikes_attempted,
        r2_leg_sig_strikes_landed, r2_leg_sig_strikes_attempted, r2_distance_sig_strikes_landed, r2_distance_sig_strikes_attempted,
        r2_clinch_sig_strikes_landed, r2_clinch_sig_strikes_attempted, r2_ground_sig_strikes_landed, r2_ground_sig_strikes_attempted,
        r3_knockdowns, r3_sig_strikes_landed, r3_sig_strikes_attempted, r3_total_strikes_landed, r3_total_strikes_attempted,
        r3_takedowns_landed, r3_takedowns_attempted, r3_subs_attempted, r3_reversals, r3_control_time,
        r3_head_sig_strikes_landed, r3_head_sig_strikes_attempted, r3_body_sig_strikes_landed, r3_body_sig_strikes_attempted,
        r3_leg_sig_strikes_landed, r3_leg_sig_strikes_attempted, r3_distance_sig_strikes_landed, r3_distance_sig_strikes_attempted,
        r3_clinch_sig_strikes_landed, r3_clinch_sig_strikes_attempted, r3_ground_sig_strikes_landed, r3_ground_sig_strikes_attempted,
        r4_knockdowns, r4_sig_strikes_landed, r4_sig_strikes_attempted, r4_total_strikes_landed, r4_total_strikes_attempted,
        r4_takedowns_landed, r4_takedowns_attempted, r4_subs_attempted, r4_reversals, r4_control_time,
        r4_head_sig_strikes_landed, r4_head_sig_strikes_attempted, r4_body_sig_strikes_landed, r4_body_sig_strikes_attempted,
        r4_leg_sig_strikes_landed, r4_leg_sig_strikes_attempted, r4_distance_sig_strikes_landed, r4_distance_sig_strikes_attempted,
        r4_clinch_sig_strikes_landed, r4_clinch_sig_strikes_attempted, r4_ground_sig_strikes_landed, r4_ground_sig_strikes_attempted,
        r5_knockdowns, r5_sig_strikes_landed, r5_sig_strikes_attempted, r5_total_strikes_landed, r5_total_strikes_attempted,
        r5_takedowns_landed, r5_takedowns_attempted, r5_subs_attempted, r5_reversals, r5_control_time,
        r5_head_sig_strikes_landed, r5_head_sig_strikes_attempted, r5_body_sig_strikes_landed, r5_body_sig_strikes_attempted,
        r5_leg_sig_strikes_landed, r5_leg_sig_strikes_attempted, r5_distance_sig_strikes_landed, r5_distance_sig_strikes_attempted,
        r5_clinch_sig_strikes_landed, r5_clinch_sig_strikes_attempted, r5_ground_sig_strikes_landed, r5_ground_sig_strikes_attempted]

    dfCheck.loc[len(dfCheck)] = performance



In [253]:
dfCheck.head()

,fighter_id,corner,name,alias,opponent_id,event_name,event_date,method_of_victory,winner_id,height,...,r5_body_sig_strikes_landed,r5_body_sig_strikes_attempted,r5_leg_sig_strikes_landed,r5_leg_sig_strikes_attempted,r5_distance_sig_strikes_landed,r5_distance_sig_strikes_attempted,r5_clinch_sig_strikes_landed,r5_clinch_sig_strikes_attempted,r5_ground_sig_strikes_landed,r5_ground_sig_strikes_attempted
0,f0a63a6f-5af0-47af-b263-bb4b1e27f698,Blue,Bo Nickal,,cc9f1178-9b80-4a78-a523-7a70d9a57640,UFC Fight Night: Sandhagen vs. Figueiredo,"May 03, 2025",KO/TKO Knee,cc9f1178-9b80-4a78-a523-7a70d9a57640,"6' 1""",...,None,None,None,None,None,None,None,None,None,None
1,f0a63a6f-5af0-47af-b263-bb4b1e27f698,Red,Bo Nickal,,0f0def84-f76e-47bc-8db4-07b679dd9e6b,UFC 309: Jones vs. Miocic,"Nov 16, 2024",U-DEC,f0a63a6f-5af0-47af-b263-bb4b1e27f698,"6' 1""",...,None,None,None,None,None,None,None,None,None,None
2,f0a63a6f-5af0-47af-b263-bb4b1e27f698,Red,Bo Nickal,,aa48edc5-2a4f-49b9-a33d-0d112abaa366,UFC 300: Pereira vs. Hill,"Apr 13, 2024",SUB Rear Naked Choke,f0a63a6f-5af0-47af-b263-bb4b1e27f698,"6' 1""",...,None,None,None,None,None,None,None,None,None,None
3,f0a63a6f-5af0-47af-b263-bb4b1e27f698,Red,Bo Nickal,,53b3e7d2-ae7b-484f-97eb-b6714e793cbd,UFC 290: Volkanovski vs. Rodriguez,"Jul 08, 2023",KO/TKO Punches,f0a63a6f-5af0-47af-b263-bb4b1e27f698,"6' 1""",...,None,None,None,None,None,None,None,None,None,None
4,f0a63a6f-5af0-47af-b263-bb4b1e27f698,Red,Bo Nickal,,a4779e0c-9c62-45f2-9e8e-6e844860a11a,UFC 285: Jones vs. Gane,"Mar 04, 2023",SUB Arm Triangle,f0a63a6f-5af0-47af-b263-bb4b1e27f698,"6' 1""",...,None,None,None,None,None,None,None,None,None,None


In [255]:
for column in dfCheck.columns:
    print(f'{column}: {dfCheck.loc[5, column]}')

KeyError: 5